In [1]:
# 导入模型
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression

In [2]:
# 各种分类模型
classifiers = [
    # K-近邻算法(KNN)：根据邻居判断类型。
    # 如果一个样本在特征空间中有K个最相似（距离相近）的样本大多数属于一个类别，则该样品也属于这个类别。
    # 默认变化是线性的，对异常点脆弱。
    KNeighborsClassifier(3), 
    # SVM=Support Vector Machine 是支持向量。
    # SVC=Support Vector Classification就是支持向量机用于分类。
    # SVC=Support Vector Regression.就是支持向量机用于回归分析。
    # 重点是边缘的支持向量，和，核函数处理非线性。
    SVC(probability=True),
    # 决策树
    # 通过对不同特征节点提问而进行分类。
    DecisionTreeClassifier(),
    # 随机森林
    # 有很多独立的决策树用bargging方式组成的模型融合系统。
    # 每棵树随机学习数据的不同特征从而产生一定的独立性。
    RandomForestClassifier(),
    # 本质是boosting模型融合
    # boosting通过上一次训练结果，提升数据权重，训练下一个模型，然后吧所有模型ensamble。
    # 理论上任何学习器都可以用于Adaboost。
    # 但一般来说，使用最广泛的Adaboost弱学习器是决策树和神经网络。
    # 对于决策树，Adaboost分类用了CART分类树，而Adaboost回归用了CART回归树。
    AdaBoostClassifier(),
    # GBDT
    # scikit-learn里没有xgboost，所以GBDT是skelearn里能用最好的boosting。
    # 在AdaBoost中，base learner是classfication tree，即f(x) = 1 or -1
    # 在GBDT中，base learner是regression tree，f(x)为一个score，可以为-0.3，可以为0.2，可以为1
    # 在XGBoost中，base learner是regression tree，f(x)为一个score，可以为-0.3，可以为0.2，可以为1
    # XGBoost引入了多种方式防止过拟合。
    # XGBoost确实引入多有意思的东西，比如parallization, column block，sampling, dart等
    GradientBoostingClassifier(),
    # 连续型朴素贝叶斯
    # GaussianNB 实现了运用于分类的高斯朴素贝叶斯算法。特征的可能性(即概率)假设为高斯分布。
    # 贝叶斯的算法我是接受的，但是特征分布假设不一定是高斯的。
    GaussianNB(),
    # 线性决策平面：LDA
    # 因为他们可以很容易计算得到闭式解(即解析解)，其天生具有多分类的特性，在实践中已经被证明很有效，并且无需调参。
    # LDA 和 QDA 都是源于简单的概率模型，这些模型对于每一个类别 k 的相关分布P(X|y=k) 都可以通过贝叶斯定理所获得。
    LinearDiscriminantAnalysis(),
    # 二次决策平面：QDA
    # 但是他们的模型复杂度是不够的，所以还是有很大局限性。
    # LDA 和 QDA 都是源于简单的概率模型，这些模型对于每一个类别 k 的相关分布P(X|y=k) 都可以通过贝叶斯定理所获得。
    QuadraticDiscriminantAnalysis(),
    # 罗辑回归
    # scikit-learn 中 logistic 回归在 LogisticRegression 类中实现了二分类（binary）、一对多分类（one-vs-rest）及多项式 logistic 回归，并带有可选的 L1 和 L2 正则化。
    # 虽然名字里有 “回归” 二字，但实际上是解决分类问题的一类线性模型。
    # 最小化目标函数loss function。
    # 实现了这些优化算法: liblinear， newton-cg， lbfgs， sag 和 saga。
    LogisticRegression()
]

In [ ]:
# 
log_cols = ["Classifier", "Accuracy"]
log = pd.DataFrame(columns=log_cols)
# 随机划分
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=0)

X = train[0::, 1::]
y = train[0::, 0]

acc_dict = {}

# 训练并得到结果
# 没有做交叉验证等工作，只是个base
for train_index, test_index in sss.split(X, y):
        # 训练，测试，特征数据，标签数据
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # 开始训练
        for clf in classifiers:
            name = clf.__class__.__name__ # 模型类名
            clf.fit(X_train, y_train) # 一句话就是训练
            train_predictions = clf.predict(X_test) # 得到预测结果
            acc = accuracy_score(y_test, train_predictions) # 计算预测精度
            
            # 放到acc_dict里
            if name in acc_dict:
                acc_dict[name] += acc
            else:
                acc_dict[name] = acc

# 根据结果画图
for clf in acc_dict:
        acc_dict[clf] = acc_dict[clf] / 10.0
        log_entry = pd.DataFrame([[clf, acc_dict[clf]]], columns=log_cols)
        log = log.append(log_entry)

plt.xlabel('Accuracy')
plt.title('Classifier Accuracy')

sns.set_color_codes("muted")
sns.barplot(x='Accuracy', y='Classifier', data=log, color="b")